In [2]:
# Installing required libraries if not already installed
%pip install requests
%pip install textblob
import textblob
from textblob import TextBlob
from pprint import pprint
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
import json

def prepare_browser():
    # Initializing Chrome options
    chrome_options = webdriver.ChromeOptions()
    # Specifying the settings needed for Selenium-Stealth
    chrome_options.add_argument("start-maximized")
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option('useAutomationExtension', False)

    # Creating the Chrome browser with the options we've set earlier
    driver = webdriver.Chrome(options=chrome_options)
    return driver

def parse_data(username, user_data):
    # Initializing a list to store post captions
    captions = []

    # Checking if there are posts in the data
    if len(user_data['edge_owner_to_timeline_media']['edges']) > 0:
        for node in user_data['edge_owner_to_timeline_media']['edges']:
            # Checking if a caption is available for the post
            if len(node['node']['edge_media_to_caption']['edges']) > 0:
                if node['node']['edge_media_to_caption']['edges'][0]['node']['text']:
                    captions.append(
                        node['node']['edge_media_to_caption']['edges'][0]['node']['text']
                    )
    
        
               
    
    for caption in captions :
        caption_blob = TextBlob(caption)
        caption_sentiment=caption_blob.sentiment.polarity   
        
        def interpret_sentiment(polarity):
            if polarity > 0:
                return "Positive"
            elif polarity < 0:
                return "Negative"
            else:
                return "Neutral"
        print("Caption:", caption)

        print("Sentiment:", interpret_sentiment(caption_sentiment))
# Populating the 'output' dictionary with user data
    locations = []  # Initializing an empty list to store locations
    likes = []  # Initializing an empty list to store like data
    comments = []  # Initializing an empty list to store comments data
    urlscount =[]
    urls = []  # Initializing an empty list to store video URLs
    owners = []  # Initializing an empty list to store owner data

    for edge in user_data['edge_felix_video_timeline']['edges']:
        urls.append(edge['node']['video_url'])
        if edge['node']:
            if 'location' in edge['node'] and edge['node']['location'] and 'name' in edge['node']['location']:
                locations.append(edge['node']['location']['name'])
            if 'edge_liked_by' in edge['node']:
                likes.append(edge['node']['edge_liked_by']['count'])
            if 'edge_media_to_comment' in edge['node']:
                comments.append(edge['node']['edge_media_to_comment']['count'])
           
            
            if 'owner' in edge['node']:
                owners.append(edge['node']['owner']['username'])

    # Now 'locations', 'likes', 'comments', 'captions', 'urls', and 'owners' will contain data for each edge

    output[username] = {
        'name': user_data['full_name'],
        'category': user_data['category_name'],
        'followers': user_data['edge_followed_by']['count'],
        'biography':user_data['biography'],
        'location': locations,
        'like':likes,
        'comments':comments,
        'caption':captions ,
        'url':urls,
        'owner':owners,
        "Sentiment": interpret_sentiment(caption_sentiment)
        
    }
    

def scrape(username, chrome):
    url = f'https://instagram.com/{username}/?__a=1&__d=dis'
    chrome.get(url)
    print(f"Attempting: {chrome.current_url}")

    if "login" in chrome.current_url:
        print("Failed/redirect to login")
        chrome.quit()
    else:
        print("Success")
        resp_body = chrome.find_element(By.TAG_NAME, "body").text
        data_json = json.loads(resp_body)
        user_data = data_json['graphql']['user']
        parse_data(username, user_data)
        chrome.quit()



def main():
    # List of usernames to scrape
    usernames = ["jlo","bellahadid","taylorswift"]

    # Defining the proxy as a string
    proxy = "server:8080"  

    for username in usernames:
        # Calling prepare_browser() with the proxy argument
        chrome = prepare_browser()
        scrape(username, chrome)
        chrome.quit()

if __name__ == '__main__':
    output = {}  # Initializing the output dictionary
    main()
    pprint(output)


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Attempting: https://www.instagram.com/jlo/?__a=1&__d=dis
Success
Caption: High Tea Kiki ☕️
Sentiment: Positive
Caption: Introducing my @IntimissimiOfficial THIS IS ME…NOW collection ✨ From Verona to Hollywood and made with love 💕 📸 @NormanJeanRoy #linkinbio
Sentiment: Positive
Caption: ✨✨✨
Sentiment: Neutral
Caption: Limitless. @SiaMusic
Sentiment: Neutral
Caption: ☀️
Sentiment: Neutral
Caption: Congratulations Tracy 💕
Sentiment: Neutral
Caption: Feels like fall 🍁 @intimissimiofficial
📸 @NormanJeanRoy
Sentiment: Neutral
Caption: 🤠
Sentiment: Neutral
Caption: 😂😜
Sentiment: Neutral
Caption: RL 🐎 NYFW
Sentiment: Neutral
Caption: TRIGGER WARNING: 
Singing + @Delola = LOL Overload! 
Happy Friday Everybody ❤️
Sentiment: Positive
Caption: Meta moment… LOL…
COACH #NYFW #CoachSpring24 #CoachNY @StuartVevers @voguemagazine #AnnaWintour
Sentiment: Neutral
Attempting: https://www.instagram.com/bellahadid/?__a=1&__d=dis
Success
Caption: Proud to be included in the 2023 #TIME100 @Time list of the mo

In [1]:
import pandas as pd

data_list = []
for username, user_data in output.items():
    # Determining the maximum length of all the lists in user_data
    max_len = max(len(user_data['like']), len(user_data['comments']), len(user_data['caption']), len(user_data['url']), len(user_data['owner']))
    
    for i in range(max_len):
        data_list.append({
            'Username': username,
            'Name': user_data['name'],
            'Category': user_data['category'],
            'Followers': user_data['followers'],
            'Biography': user_data['biography'],
            'Location': ",".join(user_data['location']),
            'Likes': user_data['like'][i] if i < len(user_data['like']) else None,
            'Comments': user_data['comments'][i] if i < len(user_data['comments']) else None,
            'Caption': user_data['caption'][i] if i < len(user_data['caption']) else None,
            'URL': user_data['url'][i] if i < len(user_data['url']) else None,
            'Owner': user_data['owner'][i] if i < len(user_data['owner']) else None,
            "Sentiment": user_data['Sentiment']
        })

df = pd.DataFrame(data_list)
excel_filename = 'instagram_data.xlsx'


NameError: name 'output' is not defined

In [4]:
df


,Username,Name,Category,Followers,Biography,Location,Likes,Comments,Caption,URL,Owner,Sentiment
0,jlo,Jennifer Lopez,Artiste,252659222,This Is Me…Now 💚,"Regency Village and Bruin Theatres - Westwood,...",308153,2902,High Tea Kiki ☕️,https://instagram.ftun13-1.fna.fbcdn.net/v/t50...,jlo,Neutral
1,jlo,Jennifer Lopez,Artiste,252659222,This Is Me…Now 💚,"Regency Village and Bruin Theatres - Westwood,...",155105,1914,Introducing my @IntimissimiOfficial THIS IS ME...,https://instagram.ftun13-1.fna.fbcdn.net/v/t50...,jlo,Neutral
2,jlo,Jennifer Lopez,Artiste,252659222,This Is Me…Now 💚,"Regency Village and Bruin Theatres - Westwood,...",428560,4439,✨✨✨,https://instagram.ftun13-1.fna.fbcdn.net/v/t50...,jlo,Neutral
3,jlo,Jennifer Lopez,Artiste,252659222,This Is Me…Now 💚,"Regency Village and Bruin Theatres - Westwood,...",272458,3271,Limitless. @SiaMusic,https://instagram.ftun13-1.fna.fbcdn.net/v/t50...,jlo,Neutral
4,jlo,Jennifer Lopez,Artiste,252659222,This Is Me…Now 💚,"Regency Village and Bruin Theatres - Westwood,...",394701,5372,☀️,https://instagram.ftun13-1.fna.fbcdn.net/v/t50...,jlo,Neutral
5,jlo,Jennifer Lopez,Artiste,252659222,This Is Me…Now 💚,"Regency Village and Bruin Theatres - Westwood,...",552447,11468,Congratulations Tracy 💕,https://instagram.ftun13-1.fna.fbcdn.net/v/t50...,jlo,Neutral
6,jlo,Jennifer Lopez,Artiste,252659222,This Is Me…Now 💚,"Regency Village and Bruin Theatres - Westwood,...",424415,8277,Feels like fall 🍁 @intimissimiofficial\n📸 @Nor...,https://instagram.ftun13-1.fna.fbcdn.net/v/t50...,jlo,Neutral
7,jlo,Jennifer Lopez,Artiste,252659222,This Is Me…Now 💚,"Regency Village and Bruin Theatres - Westwood,...",763702,15101,🤠,https://instagram.ftun13-1.fna.fbcdn.net/v/t50...,jlo,Neutral
8,jlo,Jennifer Lopez,Artiste,252659222,This Is Me…Now 💚,"Regency Village and Bruin Theatres - Westwood,...",398548,17250,😂😜,https://instagram.ftun13-1.fna.fbcdn.net/v/t50...,jlo,Neutral
9,jlo,Jennifer Lopez,Artiste,252659222,This Is Me…Now 💚,"Regency Village and Bruin Theatres - Westwood,...",582185,11085,RL 🐎 NYFW,https://instagram.ftun13-1.fna.fbcdn.net/v/t50...,jlo,Neutral
